In [1]:
import json as js
import pandas as pd
from datasets import load_dataset, Dataset
from tqdm import tqdm
tqdm.pandas()
from ast import literal_eval

In [2]:
with open("meta_infos.json", "r") as f:
    meta_infos = js.load(f)

In [3]:
template_list = list()

for task, languages in meta_infos["task_language_mapping"].items():
    if len(languages)>1:
        languages.append('all')
    for lang in languages:
        for split in ['train', 'validation', 'test']:
            template_list.append({'finetuning_task':task, 'split':split, 'language': lang})
template = pd. DataFrame(template_list)
#template = template[template.finetuning_task.str.contains('multi_eurlex')==False] # multi_eurlex we need to perform language split

In [4]:
def split_into_languages(dataset):
    dataset_new = list()

    dataset_df = pd.DataFrame(dataset)

    for item in dataset_df.to_dict(orient='records'):
        labels = item['label']
        for language, document in literal_eval(item['input']).items():
            if document is not None:
                item_new = dict()
                item_new['language'] = language
                item_new['input'] = str(document)
                item_new['label'] = labels
                dataset_new.append(item_new)

    dataset_new = pd.DataFrame(dataset_new)

    dataset_new = Dataset.from_pandas(dataset_new)
    
    return dataset_new

In [5]:
def get_sample_size(finetuning_task, split, language):
    dataset = load_dataset("joelito/lextreme", finetuning_task, split=split)
    
    if 'multi_eurlex' in finetuning_task:
        dataset = split_into_languages(dataset)
    
    if language=="all":
        return len(dataset['input'])
    else:
        dataset = pd.DataFrame(dataset)
        dataset = dataset[dataset.language==language]
        return dataset.shape[0]

In [ ]:
template['number_of_samples'] = template.progress_apply(lambda x: get_sample_size(x["finetuning_task"], x["split"], x["language"]), axis=1)

  7%|██▉                                       | 31/441 [01:57<23:14,  3.40s/it]Found cached dataset lextreme (/Users/test/.cache/huggingface/datasets/joelito___lextreme/online_terms_of_service_unfairness_levels/1.0.0/43e5402e22c4ffbf0de470c8e84115dfed02ba82d1f3dad643bbf67fd9b8b415)


 14%|█████▋                                    | 60/441 [03:26<19:29,  3.07s/it]Found cached dataset lextreme (/Users/test/.cache/huggingface/datasets/joelito___lextreme/online_terms_of_service_clause_topics/1.0.0/43e5402e22c4ffbf0de470c8e84115dfed02ba82d1f3dad643bbf67fd9b8b415)


 21%|███████▊                              | 91/441 [33:10<18:57:29, 195.00s/it]Found cached dataset lextreme (/Users/test/.cache/huggingface/datasets/joelito___lextreme/multi_eurlex_level_1/1.0.0/43e5402e22c4ffbf0de470c8e84115dfed02ba82d1f3dad643bbf67fd9b8b415)


 28%|█████████▋                         | 122/441 [3:01:27<31:29:44, 355.44s/it]Found cached dataset lextreme (/Users/test/.cache/huggingface/datasets/joelito___lextreme/multi_eurlex_level_1/1.0.0/43e5402e22c4ffbf0de470c8e84115dfed02ba82d1f3dad643bbf67fd9b8b415)


 35%|████████████▏                      | 153/441 [5:18:47<20:56:57, 261.86s/it]Found cached dataset lextreme (/Users/test/.cache/huggingface/datasets/joelito___lextreme/multi_eurlex_level_1/1.0.0/43e5402e22c4ffbf0de470c8e84115dfed02ba82d1f3dad643bbf67fd9b8b415)


...

Found cached dataset multi_eurlex (/Users/test/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/e50b15921a24b8e33941240fe19542e492392357f145d933bd497812181d610d)


Generating test split: 0 examples [00:00, ? examples/s]

Found cached dataset multi_eurlex (/Users/test/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/e50b15921a24b8e33941240fe19542e492392357f145d933bd497812181d610d)


Dataset lextreme downloaded and prepared to /Users/test/.cache/huggingface/datasets/joelito___lextreme/multi_eurlex_level_2/1.0.0/43e5402e22c4ffbf0de470c8e84115dfed02ba82d1f3dad643bbf67fd9b8b415. Subsequent calls will reuse this data.


 36%|████████████▋                      | 160/441 [5:50:40<18:52:39, 241.85s/it]Found cached dataset lextreme (/Users/test/.cache/huggingface/datasets/joelito___lextreme/multi_eurlex_level_2/1.0.0/43e5402e22c4ffbf0de470c8e84115dfed02ba82d1f3dad643bbf67fd9b8b415)


In [ ]:
template.to_excel('/Users/test/Downloads/overview_sizes.xlsx', index=False)